In [1]:
#import dependencies
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from imblearn.over_sampling import  SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
#Read in csv
file_path = '..\data\cleaned_stroke_dataset.csv'

stroke_df = pd.read_csv(file_path)

stroke_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1


In [3]:
stroke_df.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,3565.000000,3565.000000,3565.000000,3565.000000,3565.000000,3565.000000
mean,48.860309,0.125105,0.063955,108.972376,30.237406,0.056662
std,18.873140,0.330885,0.244707,48.295550,7.156210,0.231228
min,10.000000,0.000000,0.000000,55.120000,11.500000,0.000000
25%,34.000000,0.000000,0.000000,77.460000,25.400000,0.000000
50%,50.000000,0.000000,0.000000,92.650000,28.893237,0.000000
75%,63.000000,0.000000,0.000000,116.670000,33.800000,0.000000
max,82.000000,1.000000,1.000000,271.740000,92.000000,1.000000


In [4]:
stroke_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3565 entries, 0 to 3564
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             3565 non-null   object 
 1   age                3565 non-null   float64
 2   hypertension       3565 non-null   int64  
 3   heart_disease      3565 non-null   int64  
 4   ever_married       3565 non-null   object 
 5   work_type          3565 non-null   object 
 6   Residence_type     3565 non-null   object 
 7   avg_glucose_level  3565 non-null   float64
 8   bmi                3565 non-null   float64
 9   smoking_status     3565 non-null   object 
 10  stroke             3565 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 306.5+ KB


In [5]:
stroke_df.isnull().sum(axis = 0)

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [6]:
# def create_histogram(column):
#     fig = px.histogram(stroke_df, x=column)
#     fig.show()

# TODO: add legends, change colors if possible
for column in stroke_df.columns:
    fig = px.histogram(stroke_df, x=column)
    fig.show()
    

In [7]:
encoded_df = pd.get_dummies(data = stroke_df, columns = ['gender', 'work_type', 'Residence_type', 'smoking_status', 'ever_married'])
encoded_df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,ever_married_No,ever_married_Yes
0,67.0,0,1,228.69,36.600000,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1
1,61.0,0,0,202.21,28.893237,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1
2,80.0,0,1,105.92,32.500000,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1
3,49.0,0,0,171.23,34.400000,1,1,0,0,0,1,0,0,0,1,0,0,1,0,1
4,79.0,1,0,174.12,24.000000,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1


In [8]:
#TODO:add title, axes
fig = px.imshow(encoded_df.corr(), text_auto=True)
fig.layout.height = 1000
fig.layout.width = 1000
fig.show()

In [9]:

def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = encoded_df.columns
    for i in range(0, encoded_df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_correlations(df):
    au_corr = encoded_df.corr().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr

corrplot = get_correlations(encoded_df.columns)

new_line = '\n     '
print(f"The top five variables mostly correlated to one another are {new_line}{corrplot.head()}")

The top five variables mostly correlated to one another are 
     age  ever_married_Yes           0.523164
     work_type_Self-employed    0.299888
     hypertension               0.269597
     heart_disease              0.264269
     stroke                     0.250765
dtype: float64


In [10]:
corr = encoded_df.corr()["stroke"].sort_values(ascending=True)
corr.drop(['stroke'], inplace=True)
fig = px.bar(x=corr, y=corr.index, orientation='h')
fig.show()

In [11]:
fig2 = px.histogram(data_frame=stroke_df,
                    x = 'age',
                    color = 'stroke',
                    barmode = 'overlay')
fig2.layout.height = 500
fig2.layout.width = 1000
fig2.show()

In [12]:

fig3 = px.histogram(stroke_df,
                    x = 'gender',
                    y = 'stroke',
                    barmode = 'group')
fig3.layout.height = 500
fig3.layout.width = 500
fig3.show()

In [13]:
###uneven distribution for stroke datapoints
stroke_df['stroke'].value_counts(normalize= True)

0    0.943338
1    0.056662
Name: stroke, dtype: float64

Preprocessing

In [14]:
y = encoded_df['stroke']
X = encoded_df.drop(columns = 'stroke')


In [15]:
# X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=1)
# y.value_counts(normalize = True)

In [16]:
###
sm = SMOTE(random_state = 1)
X_resampled, y_resampled = sm.fit_resample(X, y)

In [17]:
y_resampled.value_counts(normalize = True)

1    0.5
0    0.5
Name: stroke, dtype: float64

In [18]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_resampled)
X_scaled = X_scaler.transform(X_resampled)

In [19]:
len(X_scaled)
len(y_resampled)

6726

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resampled, stratify=y_resampled, random_state=1)

In [21]:
#Naive-Bayes method

gnb = GaussianNB()
cv = cross_val_score(gnb, X_train, y_train, cv = 5)
print(cv)
print(cv.mean())

[0.67393459 0.66402379 0.67690783 0.64519326 0.6765873 ]
0.6673293532807903


In [22]:
#Predicting with test data and assessing with confusion matrix

from sklearn.metrics import confusion_matrix
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[302, 539],
       [ 26, 815]], dtype=int64)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.36      0.52       841
           1       0.60      0.97      0.74       841

    accuracy                           0.66      1682
   macro avg       0.76      0.66      0.63      1682
weighted avg       0.76      0.66      0.63      1682



In [24]:
lr = LogisticRegression(max_iter = 1000)
cv = cross_val_score(lr, X_train, y_train.values, cv = 5)
print(cv)
print(cv.mean())

[0.93756194 0.95044599 0.94846383 0.96432111 0.94543651]
0.9492458744317019


In [25]:
lr.fit(X_train, y_train)
y_pred2 = lr.predict(X_test)
confusion_matrix(y_test, y_pred2)

array([[835,   6],
       [ 71, 770]], dtype=int64)

In [26]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96       841
           1       0.99      0.92      0.95       841

    accuracy                           0.95      1682
   macro avg       0.96      0.95      0.95      1682
weighted avg       0.96      0.95      0.95      1682



In [27]:
dt = tree.DecisionTreeClassifier(random_state=1)
cv = cross_val_score(dt, X_train, y_train.values, cv = 5)
print(cv)
print(cv.mean())

[0.92864222 0.92368682 0.94152626 0.93657086 0.9156746 ]
0.9292201535387858


In [28]:
dt.fit(X_train, y_train)
y_pred3 = dt.predict(X_test)
confusion_matrix(y_test, y_pred3)

array([[774,  67],
       [ 37, 804]], dtype=int64)

In [29]:
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.95      0.92      0.94       841
           1       0.92      0.96      0.94       841

    accuracy                           0.94      1682
   macro avg       0.94      0.94      0.94      1682
weighted avg       0.94      0.94      0.94      1682



In [30]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn, X_train, y_train.values, cv = 5)
print(cv)
print(cv.mean())

[0.94251734 0.95044599 0.94945491 0.95341923 0.95138889]
0.9494452703446757


In [31]:
knn.fit(X_train, y_train)
y_pred4 = knn.predict(X_test)
confusion_matrix(y_test, y_pred4)

array([[818,  23],
       [ 46, 795]], dtype=int64)

In [32]:
print(classification_report(y_test, y_pred4))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       841
           1       0.97      0.95      0.96       841

    accuracy                           0.96      1682
   macro avg       0.96      0.96      0.96      1682
weighted avg       0.96      0.96      0.96      1682



In [33]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf, X_train, y_train.values, cv = 5)
print(cv)
print(cv.mean())

[0.96035679 0.95837463 0.97224975 0.96531219 0.9593254 ]
0.9631237513175075


In [34]:
rf.fit(X_train, y_train)
y_pred5 = rf.predict(X_test)
confusion_matrix(y_test, y_pred5)

array([[835,   6],
       [ 46, 795]], dtype=int64)

In [35]:
print(classification_report(y_test, y_pred5))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       841
           1       0.99      0.95      0.97       841

    accuracy                           0.97      1682
   macro avg       0.97      0.97      0.97      1682
weighted avg       0.97      0.97      0.97      1682



In [36]:
svc = SVC(probability=True)
cv = cross_val_score(svc, X_train, y_train.values, cv = 5)
print(cv)
print(cv.mean())

[0.9395441  0.95044599 0.95044599 0.96729435 0.95238095]
0.9520222757090944


In [37]:
svc.fit(X_train, y_train)
y_pred6 = svc.predict(X_test)
confusion_matrix(y_test, y_pred6)

array([[841,   0],
       [ 70, 771]], dtype=int64)

In [38]:
print(classification_report(y_test, y_pred6))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       841
           1       1.00      0.92      0.96       841

    accuracy                           0.96      1682
   macro avg       0.96      0.96      0.96      1682
weighted avg       0.96      0.96      0.96      1682



In [39]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [('lr', lr), ('knn', knn), ('rf', rf),('gnb', gnb), ('svc', svc)], voting = 'soft')

cv = cross_val_score(voting_clf, X_train, y_train.values, cv = 5)
print(cv)
print(cv.mean())

[0.95936571 0.95242815 0.96134787 0.96432111 0.95535714]
0.9585639954693473


In [40]:
voting_clf.fit(X_train, y_train)
y_pred7 = voting_clf.predict(X_test)
confusion_matrix(y_test, y_pred7)

array([[830,  11],
       [ 46, 795]], dtype=int64)

In [41]:
print(classification_report(y_test, y_pred7))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       841
           1       0.99      0.95      0.97       841

    accuracy                           0.97      1682
   macro avg       0.97      0.97      0.97      1682
weighted avg       0.97      0.97      0.97      1682



In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [43]:
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [55]:
lr = LogisticRegression()
param_grid = {'penalty': ['l2'],
              'C': np.logspace(-4, 4, 20)}

clf_lr = GridSearchCV(lr, param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_lr = clf_lr.fit(X_train, y_train)

clf_performance(best_clf_lr, 'LogisticRegression')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
LogisticRegression
Best Score: 0.9498399326694669
Best Parameters: {'C': 0.08858667904100823, 'penalty': 'l2'}


In [57]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors' : list(range(1,20)),
              'leaf_size' : list(range(1, 20)),
              'p' : [1,2]}
clf_knn = GridSearchCV(knn, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_knn = clf_knn.fit(X_train,y_train)
clf_performance(best_clf_knn,'KNN')

Fitting 5 folds for each of 722 candidates, totalling 3610 fits
KNN
Best Score: 0.9601509037708246
Best Parameters: {'leaf_size': 1, 'n_neighbors': 2, 'p': 2}
